# Trading Bot Performance Analysis

This notebook analyzes the performance of the insider trading bot.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import sys
from pathlib import Path

# Add project root to path
sys.path.append(str(Path.cwd().parent))

from src.utils.database import Database
from src.data_collection.price_fetcher import PriceFetcher

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

%matplotlib inline

## 1. Load Data

In [ ]:
# Connect to database
db = Database()

# Load all trades
trades = db.get_all_trades()

# Convert to DataFrame
trades_df = pd.DataFrame([{
    'ticker': t.ticker,
    'entry_date': t.entry_date,
    'entry_price': t.entry_price,
    'exit_date': t.exit_date,
    'exit_price': t.exit_price,
    'shares': t.shares,
    'exit_reason': t.exit_reason,
    'net_pnl': t.net_pnl,
    'return_pct': t.return_pct,
    'status': t.status
} for t in trades])

print(f"Total trades: {len(trades_df)}")
print(f"Open positions: {len(trades_df[trades_df['status'] == 'OPEN'])}")
print(f"Closed positions: {len(trades_df[trades_df['status'] == 'CLOSED'])}")

trades_df.head()

## 2. Overall Performance Metrics

In [ ]:
# Filter closed trades
closed_trades = trades_df[trades_df['status'] == 'CLOSED'].copy()

if len(closed_trades) > 0:
    # Calculate metrics
    total_trades = len(closed_trades)
    winning_trades = len(closed_trades[closed_trades['net_pnl'] > 0])
    losing_trades = len(closed_trades[closed_trades['net_pnl'] <= 0])
    win_rate = winning_trades / total_trades * 100
    
    total_pnl = closed_trades['net_pnl'].sum()
    avg_win = closed_trades[closed_trades['net_pnl'] > 0]['net_pnl'].mean()
    avg_loss = closed_trades[closed_trades['net_pnl'] <= 0]['net_pnl'].mean()
    avg_return = closed_trades['return_pct'].mean() * 100
    
    # Display metrics
    print("="*50)
    print("PERFORMANCE SUMMARY")
    print("="*50)
    print(f"Total Trades: {total_trades}")
    print(f"Winning Trades: {winning_trades} ({win_rate:.1f}%)")
    print(f"Losing Trades: {losing_trades}")
    print(f"\nTotal P&L: ${total_pnl:,.2f}")
    print(f"Average Win: ${avg_win:,.2f}")
    print(f"Average Loss: ${avg_loss:,.2f}")
    print(f"Average Return: {avg_return:.2f}%")
    print("="*50)
else:
    print("No closed trades yet")

## 3. P&L Distribution

In [ ]:
if len(closed_trades) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # P&L histogram
    axes[0].hist(closed_trades['net_pnl'], bins=20, edgecolor='black', alpha=0.7)
    axes[0].axvline(0, color='red', linestyle='--', linewidth=2)
    axes[0].set_xlabel('Net P&L ($)')
    axes[0].set_ylabel('Frequency')
    axes[0].set_title('Distribution of Trade P&L')
    axes[0].grid(True, alpha=0.3)
    
    # Return percentage histogram
    axes[1].hist(closed_trades['return_pct'] * 100, bins=20, edgecolor='black', alpha=0.7, color='green')
    axes[1].axvline(0, color='red', linestyle='--', linewidth=2)
    axes[1].set_xlabel('Return (%)')
    axes[1].set_ylabel('Frequency')
    axes[1].set_title('Distribution of Returns')
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## 4. Cumulative Returns

In [ ]:
if len(closed_trades) > 0:
    # Sort by exit date
    closed_trades_sorted = closed_trades.sort_values('exit_date')
    
    # Calculate cumulative P&L
    closed_trades_sorted['cumulative_pnl'] = closed_trades_sorted['net_pnl'].cumsum()
    
    # Plot
    plt.figure(figsize=(12, 6))
    plt.plot(closed_trades_sorted['exit_date'], closed_trades_sorted['cumulative_pnl'], 
             linewidth=2, marker='o', markersize=4)
    plt.axhline(0, color='red', linestyle='--', alpha=0.5)
    plt.xlabel('Date')
    plt.ylabel('Cumulative P&L ($)')
    plt.title('Cumulative Profit/Loss Over Time')
    plt.grid(True, alpha=0.3)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

## 5. Trade Analysis by Exit Reason

In [ ]:
if len(closed_trades) > 0:
    # Group by exit reason
    exit_analysis = closed_trades.groupby('exit_reason').agg({
        'net_pnl': ['count', 'sum', 'mean'],
        'return_pct': 'mean'
    }).round(2)
    
    print("\nPerformance by Exit Reason:")
    print(exit_analysis)
    
    # Visualize
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Count by exit reason
    exit_counts = closed_trades['exit_reason'].value_counts()
    axes[0].bar(exit_counts.index, exit_counts.values, edgecolor='black')
    axes[0].set_xlabel('Exit Reason')
    axes[0].set_ylabel('Number of Trades')
    axes[0].set_title('Trades by Exit Reason')
    axes[0].tick_params(axis='x', rotation=45)
    
    # Average return by exit reason
    exit_returns = closed_trades.groupby('exit_reason')['return_pct'].mean() * 100
    colors = ['green' if x > 0 else 'red' for x in exit_returns]
    axes[1].bar(exit_returns.index, exit_returns.values, color=colors, edgecolor='black')
    axes[1].axhline(0, color='black', linestyle='-', linewidth=0.5)
    axes[1].set_xlabel('Exit Reason')
    axes[1].set_ylabel('Average Return (%)')
    axes[1].set_title('Average Return by Exit Reason')
    axes[1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()

## 6. Top Performers and Worst Trades

In [ ]:
if len(closed_trades) > 0:
    print("\nTop 5 Best Trades:")
    print(closed_trades.nlargest(5, 'net_pnl')[['ticker', 'entry_date', 'exit_date', 'net_pnl', 'return_pct']])
    
    print("\nTop 5 Worst Trades:")
    print(closed_trades.nsmallest(5, 'net_pnl')[['ticker', 'entry_date', 'exit_date', 'net_pnl', 'return_pct']])

## 7. Active Signals Summary

In [ ]:
# Load active signals
signals = db.get_active_signals()

if signals:
    signals_df = pd.DataFrame([{
        'ticker': s.ticker,
        'signal_date': s.signal_date,
        'signal_strength': s.signal_strength,
        'conviction_score': s.conviction_score,
        'entry_price': s.entry_price,
        'target_position_size': s.target_position_size,
    } for s in signals])
    
    print(f"\nActive Signals: {len(signals_df)}")
    print(signals_df)
else:
    print("\nNo active signals")

In [ ]:
# Close database connection
db.close()